In [1]:
import csv
import requests
from bs4 import BeautifulSoup

### Função para acessar página, baixa código HTML e abrir um objeto BeautifulSoup

In [2]:
def prepara_sopa(url):
	# Cria objeto BeautifulSoup com o código HTML
	# url: string com o link da página
	# retorna objeto BeautifulSoup
	html = requests.get(url)
	sopa = BeautifulSoup(html.content, 'html.parser')
	return sopa

### Função para pegar lista de nomes e seus códigos

In [3]:
def dicio_menu(sopa, selector):
	# Função para pegar a lista do menu e seus valores
	# sopa: objeto BeautifulSoup
	# selector: string com seletor do menu
	# retorna: dicionário com par opção:valor
	lista_opcoes = sopa.select(selector)[0].find_all("option")
	dicio = {}
	for item in lista_opcoes:
		nome = item.text.encode('utf-8')
		valor = item.attrs['value'].encode('utf-8')
		dicio[nome] = valor
	return dicio

### Função para gerar URL usando código e data

In [4]:
def gera_url(conta, periodo):
	# Gera uma url usando o código da conta(gabinete) e o período selecionado
	# conta: string do seletor do menu "conta"
	# periodo: string do seletor do menu "periodo"
	# retorna string com URL
	url = 'http://transparencia.alesc.sc.gov.br/gabinetes.php?conta={0}&&periodo={1}'.format(conta, periodo)
	return url

### Função para extrair tabela da página e retornar dicionário com valores, incluindo gabinete e data

In [5]:
def extrai_tabela(sopa, conta, periodo):
	# Extrai tabela HTML da página e inclui conta e periodo
	# sopa: objeto BeautifulSoup
	# conta: string
	# periodo: string
	# retorna: lista - cada item é uma linha da tabela (a primeira é o cabeçalho)
	tabela_html = sopa.select('#conteudo table tr')
	tabela_final = []
	tabela_final.append(['Gabinete', 'Periodo', 'Grupo', 'Total'])

	for linha in tabela_html[1:]:
		valor_grupo = linha.select('td')[0].text.encode('utf-8')
		valor_total = linha.select('td')[1].text.strip().encode('utf-8')
		tabela_final.append([conta, periodo, valor_grupo, valor_total])

	return tabela_final

### Função para salvar lista no arquivo CSV

In [6]:
def salva_csv(tabela_final, header):
	# Salva uma lista de linhas no arquivo CSV
	# tabela_final: lista em que cada item é uma linha da tabela
	# header: booleano, indica se devemos escrever o cabeçalho ou não
	# retorna: nada
	with open('alesc_gabinetes.csv', 'a') as f:
		writer = csv.writer(f)
		if header is True:
			writer.writerow(tabela_final[0])
		writer.writerows(tabela_final[1:])
	return